## 定义模型

In [57]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.env_util import *
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

def get_completion(prompt):
    message = [{"role": "user", "content": prompt}]
    model = ChatOpenAI(
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        model_name=os.getenv("MODEL_NAME"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        temperature=0.0,
    )
    return model.invoke(message)

get_completion("你是谁")

AIMessage(content='\n\n你好！我是通义千问，阿里巴巴集团旗下的超大规模语言模型。我能够回答问题、创作文字，比如写故事、写公文、写邮件、写剧本、逻辑推理、编程等等，还能表达观点，玩游戏等。我熟练掌握多种语言，包括但不限于中文、英文、德语、法语、西班牙语等。\n\n如果你有任何问题或需要帮助，随时告诉我！😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 290, 'prompt_tokens': 12, 'total_tokens': 302, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 205, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': None}, 'model_name': 'Qwen/QwQ-32B', 'system_fingerprint': '', 'id': '01968030c05b25b7cb43503f7be9c1f6', 'finish_reason': None, 'logprobs': None}, id='run-67e9c66e-e3e6-4043-9932-d14203a02193-0', usage_metadata={'input_tokens': 12, 'output_tokens': 290, 'total_tokens': 302, 'input_token_details': {}, 'output_token_details': {'reasoning': 205}})

## Prompt 模板

In [58]:
prompt_template = """
你是一个问答机器人，你的任务是根据下述给定的已知信息回答用户问题。如果你不知道答案，就回答不知道，不要胡编乱造。

已知信息：
{context}

用户问题：
{query}

如果已知信息不包含用户问题的答案，或者已知信息不足以回答用户的问题，就回答不知道，不要胡编乱造。
请不要输出已知信息中不包含的信息或答案。
请用中文回答用户问题。
"""

def build_prompt(prompt_template, **kwargs):
    """
    自定义参数渲染 Prompt 模板
    """
    inputs = {}
    for k, v in kwargs.items():
        if isinstance(v, list) and all(isinstance(elem, str) for elem in v):
            val = "\n\n".join(v)
        else:
            val = v
        inputs[k] = val
    return prompt_template.format(**inputs)


## 向量间相似度的计算

余弦相似度通过向量点积与模长乘积的比值计算，公式如下： 

$$
\cos(\theta) = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \|\mathbf{B}\|} = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \sqrt{\sum_{i=1}^{n} B_i^2}}
$$

L2 距离是向量对应元素差的平方和的平方根，公式如下：

$$
d(\mathbf{A}, \mathbf{B}) = \sqrt{\sum_{i=1}^{n} (A_i - B_i)^2}
$$

In [59]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(A, B):
    ''' 余弦相似度（越大越相似） '''
    return dot(A, B)/(norm(A)*norm(B))

def l2_distance(A, B):
    ''' 欧式距离（越小越相似） '''
    x = np.asarray(A)
    y = np.asarray(B)
    return norm(x-y)

## 嵌入模型

[嵌入模型库](https://www.modelscope.cn/models?page=1&tasks=sentence-embedding&type=nlp)

标准

- **找需求相关的语料库来进行文本向量转换测试，进行评估**
- 大多数场景下，开源的嵌入模型使用效果都一般，要进行检索召回率，建议对模型进行微调
- 嵌入模型的维度越高，表示特征细节提取越丰富

In [62]:
from typing import List
from langchain_openai import OpenAIEmbeddings
from volcenginesdkarkruntime import Ark

def get_embeddings(texts: List[str]):
    ''' OpenAI Embeddings '''
    embedding_model = OpenAIEmbeddings(
        api_key=os.getenv('OPENAI_API_KEY'),
        base_url=os.getenv('OPENAI_BASE_URL'),
        model=os.getenv('EMBEDDING_MODEL_NAME')
    )
    return embedding_model.embed_documents(texts)

def get_ark_embeddings(texts: List[str]):
    client = Ark(api_key=os.getenv('ARK_API_KEY'))
    return client.embeddings.create(
        model="doubao-embedding-large-text-240915",
        input=texts
    )

test_texts = ['吃完海鲜可以喝牛奶吗？']
vec = get_embeddings(test_texts)[0]
print(f'Total Dimension: {len(vec)}')
print(f'Fist 10 Dimensions: {vec[:10]}')

vec = get_ark_embeddings(test_texts).data[0].embedding
print(f'\n[ARK] Total Dimension: {len(vec)}')
print(f'[ARK] Fist 10 Dimensions: {vec[:10]}')

Total Dimension: 1024
Fist 10 Dimensions: [0.05262557417154312, 0.02764252945780754, 0.07897865772247314, -0.0029994763899594545, 0.0073790643364191055, 0.026514261960983276, 0.009670855477452278, -0.0016747706104069948, 0.045372430235147476, 0.004772466607391834]

[ARK] Total Dimension: 4096
[ARK] Fist 10 Dimensions: [0.2421875, 3.859375, -0.482421875, -3.96875, 0.74609375, -2.625, 1.6015625, 3.359375, 0.3984375, 3.25]


In [63]:
query = "国际争端"
# 前两条为国际争端
documents = [
    "联合国就苏丹达尔富地区大规模暴力事件发出警告",
    "土耳其、芬兰、瑞典与北约代表将继续就瑞典“入约”问题进行谈判",
    "日本歧阜市陆上自卫队射击场内发生枪击事件 3人受伤",
    "国家游泳中心（水立方）：恢复游泳、嬉水乐园等水上项目运营",
    "我国首次在空间站开展舱外辐射生物学暴露实验"
]

# query_vector = get_embeddings([query])[0]
# doc_vectors = get_embeddings(documents)

# 实测 doubao Embedding 模型效果更好
query_vector = get_ark_embeddings([query]).data[0].embedding
doc_vectors = [doc_vector.embedding for doc_vector in get_ark_embeddings(documents).data]

# 余弦距离越大表示越相似
print(f'query 与自己的余弦距离为：{cosine_similarity(query_vector, query_vector)}')
print('query 与文档的余弦距离为：')
for doc_vector in doc_vectors:
    print(f'{cosine_similarity(query_vector, doc_vector)}')

print('*' * 80)

# L2距离（欧式距离）越小表示越相似
print(f'query 与自己的L2距离为：{l2_distance(query_vector, query_vector)}')
print('query 与文档的L2距离为：')
for doc_vector in doc_vectors:
    print(f'{l2_distance(query_vector, doc_vector)}')

query 与自己的余弦距离为：1.0
query 与文档的余弦距离为：
0.8851309597447
0.8601865743943213
0.802517564858825
0.7592321142985579
0.796052817036916
********************************************************************************
query 与自己的L2距离为：0.0
query 与文档的L2距离为：
63.85760211053751
69.9480541076191
82.65289032621875
91.29779700506289
84.24111387780202


## 向量数据库 Chroma

官方文档：https://docs.trychroma.com/docs/overview/introduction

In [59]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
import PyPDF2

def extract_text_from_pdf(filename, page_numbers=None, min_line_length=1):
    """Extract text from a PDF file.
    Args:
        filename (str): Path to the PDF file.
        page_numbers (list[int], optional): List of page numbers to extract. If None, extract all pages.
        min_line_length (int, optional): Minimum length of a line to be included in the output.
    Returns:
        str: Extracted text.
    """
    paragraphs = []
    buffer, full_text = '', ''
    # 提取全部文本
    for i, page_layout in enumerate(extract_pages(filename)):
        if page_numbers is not None and i not in page_numbers:
            continue
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                full_text += element.get_text() + '\n'
    # 按空行分隔，将文本重新组织成段落
    for line in full_text.split('\n'):
        if len(line) >= min_line_length:
            buffer += (' ' + line) if not line.endswith('-') else line.strip('-')
        elif buffer:
            paragraphs.append(buffer.strip())
            buffer = ''
    if buffer:
        paragraphs.append(buffer.strip())
    return paragraphs  

# pdfminer 的效果不太好
paragraphs = extract_text_from_pdf("file/医学史.pdf", min_line_length=45)
for paragraph in paragraphs[:6]:
    print(paragraph + '\n')   

埃及是最早出现阶级和奴隶制的国家之一。大部分有关医学的史料都记录在 “纸草文” （papyrus,—

名的卡亨（Kahum）纸草文约写于公元前 1850年，主要记载妇科资料，史密斯（Eawin  Smith）纸草文约写于公元前 1800 年，是介绍外科知识的文献；埃伯斯（George

公元前10世纪，居住于印度的雅利安人产生了婆罗门教，其经典是吠陀）。“吠陀”(veda)的意思是  “求知”或“知识”，记载了公元前2000年一公元前1000年的历史史料。雅利安文化及其医学的来源是四部《吠陀》经，即《梨俱吠陀》（Rig-veda），译作《赞诵明论》，《娑摩吠陀 》(Soma veda)，译作《歌咏明论》，《夜柔吠陀》(Yajur-veda)，译作《祭祀明论》，阿达婆吠陀）  (Atharva-veda)），译作《禳灾明论》。《梨俱吠陀》约在公元前 1500 年一公元前 900

veda)，译作《生命经》，是吠陀圣典的补充，记载了较多的医学史料，总结了对疾病的诊治经验

208）创用麻沸散施行外科手术。有医圣之称的张仲景（约 150—215，也有一说是154—

282）的《针灸甲乙经》等著作对后世产生了深远影响。本时期本草著作达70余种，最有影响的是



In [49]:
# 为了演示方便，我们只取两页（第一章/Page 3、Page 4）
paragraphs = extract_text_from_pdf(
    "file/医学史.pdf",
    page_numbers=[2, 3],
    min_line_length=10
)

for paragraph in paragraphs:
    print(paragraph + '\n')   

金元时期战争频发，疾病广泛流行，过去对病因、病机的解释和当时盛行的经方、局方等医方，已不能适应临床需要，当时一些医 家产生了“古方不能治今病”的思想。刘完素、张元素、张从正、李杲、王好古、朱震亨等医学家相继兴起，他们从实践中对医学理 论作出新的探讨，阐发各自不同认识，创立各具特色的理论学说，形成了刘完素（的110-1200）为代表的河间学派和以张元素

（生卒之年不详）为代表的易水学派，展开了学术争鸣，延续至明请两代，开拓了中医学发恩的新局面。

清前中期的医学发展局面错综复杂。一方面，中医学传统的理论和实践经过长期的历史检验和积累，至此已臻于完善和成熟，无论

是总体的理论阐述，抑或临床各分科的实际诊治方法，都已有了完备的体系，而且疗效在当时条件下是卓著的，另一方面，由于长

期的自闭守关，这一时期的医学有所停滞。

古代西方医学最具影响、代表性的是古希腊医学、古罗马医学。

1. 古希腊医学（公元前 450 年—公元前1世纪）

古希腊在公元前7世纪—公元前6世纪进入奴隶制社会。著名哲学家恩培多克勒（Empedocles，约公元前 495 年—公元前 435 年） 反对“神创造宇宙一切”的观点，提出一切物体都是由火、空气、水和土等四种元素按不同数量、比例混合而成的“四元素论”。著名 思想家亚里士多德（Aristorle，公元前 384 年—公元前 322 年）在其著作自然之阶梯）中，早已提出类似达尔文进化论的观点。

他还解剖过不少动物尸体，以图示介绍动物内脏和器官，是最早的解剖图谱的制作者。

希波克拉底被誉为西方医学之父。《希波克拉底文集》既有论述医生的道德修养、行医的经典格言，如著名的《希波克拉底哲

言》，又有对医学技术及某些疾病发病过程的详细记载，是研究古希腊医学的最重要典籍。

2. 古罗马的医学（公元前1 世纪—公元4 世纪）

公元前2世纪，罗马占领了希腊，使许多具有高超医术和丰富医学经验的希腊医生涌入罗马，他们使罗马医学有了长足的进步。

盖伦（Claudius Galenus,129-199）是古罗马的著名医学家，其医学成就仅次于希波克拉底。盖伦首次证实了脊髓的节段性功能， 他重视药物治疗，有自己的专用药房，迄今，药房制剂仍被称为“盖伦制剂”。但是他的朴素唯物主义观点中夹杂有“目的论” 的观

点，即认为自然界中的一切都是有目的的，人的构造，

In [32]:
import chromadb
from chromadb.config import Settings

class MyVectorDBConnector:
    def __init__(self, collection_name, embedding_fn):
        # 内存模式
        chroma_client = chromadb.Client(Settings(allow_reset=True))
        # 数据持久化
        # chroma_client = chromadb.PersistentClient(path="./chroma")

        # NOTE: 清空数据，为了演示，实际不需要每次 reset()，并且是不可逆的
        chroma_client.reset()

        # 创建一个 collection
        self.collection = chroma_client.get_or_create_collection(name=collection_name)
        self.embedding_fn = embedding_fn

    def add_documents(self, documents):
        '''向 collection 中添加文档与向量'''
        self.collection.add(
            embeddings=self.embedding_fn(documents),        # 每个文档的向量
            documents=documents,                            # 文档的原文
            ids=[f"id{i}" for i in range(len(documents))]   # 每个文档的 id
        )

    def search(self, query, top_n):
        '''检索向量数据库'''
        results = self.collection.query(
            query_embeddings=self.embedding_fn([query]),
            n_results=top_n
        )
        return results

**NOTE:** 这里如果提示 `sqlite` 版本过低，可以安装 `pysqlite3-binary`，然后在虚拟环境的 `chromadb/__init__.py` 中添加如下代码：
```python
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
```